<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [50]:
%%capture
!cd npmf && git pull

import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from operator import itemgetter
import pickle

import numpy as np
import math
from numpy.ma.core import outerproduct
import pandas as pd
from pandas.tseries.offsets import Day, BDay
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm


import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_dataset, put_dataset
from npmf.utils.eikon import column_mapping
from npmf.utils.dataset import get_stocks_in_timeframe, get_historic_dates, get_forecast_dates
from npmf.utils.dataset import get_global_local_column, create_fundamental_df
from npmf.utils.tests import pickle_df

In [5]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
np.random.seed(69)

# Create a Neural network class

In [8]:
class MultivariateNetwork(nn.Module):
    def __init__(self, lag_len, cat_len, out_len, hidden_dim):
        super().__init__()

        self.pre = nn.Sequential(
            nn.Linear(lag_len, hidden_dim),
            nn.ReLU(),
        )

        self.predict = nn.Sequential(
            nn.Linear(hidden_dim + cat_len, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_len),
        )


    def forward(self, lags, cats):

        x = self.pre(lags)
        x = torch.cat((x, cats), dim=1)
        x = self.predict(x)

        return x

# Get some data

In [9]:
stock_df = get_dataset("stock-oil-final:latest", project="master-test")
meta_df = get_dataset("meta-oil-final:latest", project="master-test")
fundamentals_df = get_dataset("fundamentals-oil-final:latest", project="master-test")
macro_df = get_dataset("macro-oil-final:latest", project="master-test")

stock_df = stock_df.drop_duplicates(subset=["ticker", "date"])

wandb: Currently logged in as: ankile (krankile). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact stock-oil-final:latest, 77.63MB. 1 files... Done. 0:0:0


# Define a dataset which can iterate through time 

We want to train our neural network like the person experiences the world. I.e. we have a window of time and look at recent financial reports and macro variables to predict future market capitalization. We want to train multiple epochs over one time window and validation period, in this manner we will not have any "learned future" effect which could occur if one were to have epochs run over the all time windows. 

In [59]:
class TimeDeltaDataset(Dataset):
    def _get_last_q_fundamentals(self, target, fundamental_df, q):
        for key, df in tqdm(fundamental_df.groupby(by="ticker"), desc="1/3 Get fundamentals"):

            padding = pd.DataFrame(
                np.empty((q, fundamental_df.loc[:, "revenue":].shape[1])),
                columns=df.loc[:, "revenue":].columns,
            )

            padding[:] = np.nan
            padded_df = pd.concat([padding, df.loc[:, "revenue":]], axis=0)

            target = pd.concat([target, padded_df.iloc[-q:, :]], axis=0)

        return target

    def _get_macro_normalized(self, legal_macro_df, macro_df):
        df = legal_macro_df.copy()
        for column in [
            c for c in legal_macro_df.columns if (c != "date") and ("_fx" not in c)
        ]:
            df[column] = legal_macro_df[column] / (
                int(math.ceil(macro_df[column].max() / 100.0)) * 100
            )
        return df

    def __init__(
        self,
        current_time,
        forecast_window,
        training_window,
        n_reports,
        stock_df,
        fundamental_df,
        meta_df,
        macro_df,
    ):
        historic_dates = get_historic_dates(current_time, training_window)
        forecast_dates = get_forecast_dates(current_time, forecast_window)

        legal_stock_df = stock_df.copy().loc[stock_df.date.isin(historic_dates), :]
        
        # Only keep fundamentals for where we have stock data
        stock_tickers = legal_stock_df.ticker.unique()
        
        legal_fundamental_df = fundamental_df[
            (fundamental_df.announce_date < current_time) & (fundamental_df.ticker.isin(stock_tickers))
        ]

        legal_meta_df = meta_df.set_index("ticker")
        legal_macro_df = macro_df.loc[
            macro_df.date.isin(historic_dates), :
        ]  # TODO change to current_time - stock__macro_days_lookback_days

        # Important dimensions
        n_companies_with_fundamentals = len(legal_fundamental_df.ticker.unique())
        m_fundamentals = legal_fundamental_df.loc[:, "revenue":].shape[1]

        # Get last q fundamentals and return NA rows if they are still missing
        fundamental_df_all_quarters = pd.DataFrame(
            data=np.empty((0, m_fundamentals)),
            columns=fundamental_df.loc[:, "revenue":].columns,
        )
        fundamental_df_all_quarters = self._get_last_q_fundamentals(
            fundamental_df_all_quarters, legal_fundamental_df, n_reports
        )
        fundamentals = fundamental_df_all_quarters.to_numpy().reshape(
            (n_companies_with_fundamentals, n_reports * m_fundamentals)
        )

        # Construct columns for relative information
        (
            relative_to_global_market_column,
            relative_to_current_market_column,
            last_market_cap_col,
        ) = get_global_local_column(legal_stock_df)
        

        # Create dataframe
        fundamental_df = create_fundamental_df(fundamentals, legal_fundamental_df, n_reports, relative_to_current_market_column, relative_to_global_market_column, last_market_cap_col)

        formatted_stocks = get_stocks_in_timeframe(legal_stock_df, historic_dates, scale=True, remove_na=True)
        self.stocks_and_fundamentals = formatted_stocks.join(fundamental_df)

        # Get forecasts
        forecasts = stock_df[stock_df.date.isin(forecast_dates)]

        forecasts_unormalized = get_stocks_in_timeframe(
            forecasts, forecast_dates, scale=False, remove_na=False,
        )

        # TODO: Check if using the same MinMax-scaler as for training set is better
        forecasts_normalized = forecasts_unormalized.div(last_market_cap_col, axis=0)
        self.forecast = forecasts_normalized.loc[self.stocks_and_fundamentals.index, :]

        # Join meta and stock-fundamentals
        legal_meta_df = legal_meta_df.loc[self.stocks_and_fundamentals.index, :]
        legal_meta_df.loc[:, "exchange_code":"state_province_hq"] = legal_meta_df.loc[
            :, "exchange_code":"state_province_hq"
        ].astype("category")
        legal_meta_df.loc[:, "economic_sector":"activity"] = legal_meta_df.loc[
            :, "economic_sector":"activity"
        ].astype("category")

        legal_meta_df["founding_year"] = legal_meta_df["founding_year"] / 2000
        self.meta_df = legal_meta_df

        # Ready macro data
        self.macro_df = self._get_macro_normalized(legal_macro_df, macro_df).iloc[:, 1:]

    def __len__(self):
        return self.stocks_and_fundamentals.shape[0]

    def __getitem__(self, idx):

        return (
            self.stocks_and_fundamentals.iloc[idx, :],
            self.meta_df.iloc[idx, :],
            self.macro_df.T,
        ), self.forecast.iloc[idx, :]


In [60]:
current_time = pd.to_datetime("2010-03-01")
n_reports = 4
training_window = 240
forecast_window = 20

delta_set = TimeDeltaDataset(current_time, forecast_window, training_window, n_reports, stock_df, fundamentals_df, meta_df, macro_df)

1/3 Get fundamentals:   1%|          | 5/462 [00:00<00:10, 45.68it/s]

padding
         revenue   gross_profit         ebitda           ebit     net_income  \
0   0.000000e+00  4.940656e-324   0.000000e+00  4.940656e-324  9.881313e-324   
1  2.470328e-323  5.434722e-323  5.928788e-323  6.422853e-323  6.916919e-323   
2  1.086944e-322  1.136351e-322  1.185758e-322  1.235164e-322  1.284571e-322   
3  1.679823e-322  1.729230e-322  1.778636e-322  1.828043e-322  1.877449e-322   

             fcf   total_assets  total_current_assets  total_liabilites  \
0  1.482197e-323  1.976263e-323         2.470328e-323     2.964394e-323   
1  7.410985e-323  7.905050e-323         8.399116e-323     8.893182e-323   
2  1.333977e-322  1.383384e-322         1.432790e-322     1.482197e-322   
3  1.926856e-322  1.976263e-322         2.025669e-322     2.075076e-322   

   total_current_liabilities  long_term_debt_p_assets  \
0              3.458460e-323            3.952525e-323   
1              9.387247e-323            9.881313e-323   
2              1.531604e-322            1.58

1/3 Get fundamentals:   5%|▍         | 22/462 [00:00<00:08, 49.74it/s]

padding
   revenue  gross_profit  ebitda  ebit  net_income  fcf  total_assets  \
0      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
1      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
2      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
3      NaN           NaN     NaN   NaN         NaN  NaN           NaN   

   total_current_assets  total_liabilites  total_current_liabilities  \
0                   NaN               NaN                        NaN   
1                   NaN               NaN                        NaN   
2                   NaN               NaN                        NaN   
3                   NaN               NaN                        NaN   

   long_term_debt_p_assets  short_term_debt_p_assets  gross_profit_p  \
0                      NaN                       NaN             NaN   
1                      NaN                       NaN             NaN   
2                      NaN                       

1/3 Get fundamentals:   6%|▌         | 28/462 [00:00<00:08, 49.44it/s]

...

padding
   revenue  gross_profit  ebitda  ebit  net_income  fcf  total_assets  \
0      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
1      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
2      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
3      NaN           NaN     NaN   NaN         NaN  NaN           NaN   

   total_current_assets  total_liabilites  total_current_liabilities  \
0                   NaN               NaN                        NaN   
1                   NaN               NaN                        NaN   
2                   NaN               NaN                        NaN   
3                   NaN               NaN                        NaN   

   long_term_debt_p_assets  short_term_debt_p_assets  gross_profit_p  \
0                      NaN                       NaN             NaN   
1                      NaN                       NaN             NaN   
2                      NaN                       

1/3 Get fundamentals:  99%|█████████▉| 458/462 [00:09<00:00, 46.64it/s]

padding
   revenue  gross_profit  ebitda  ebit  net_income  fcf  total_assets  \
0      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
1      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
2      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
3      NaN           NaN     NaN   NaN         NaN  NaN           NaN   

   total_current_assets  total_liabilites  total_current_liabilities  \
0                   NaN               NaN                        NaN   
1                   NaN               NaN                        NaN   
2                   NaN               NaN                        NaN   
3                   NaN               NaN                        NaN   

   long_term_debt_p_assets  short_term_debt_p_assets  gross_profit_p  \
0                      NaN                       NaN             NaN   
1                      NaN                       NaN             NaN   
2                      NaN                       

1/3 Get fundamentals: 100%|██████████| 462/462 [00:09<00:00, 48.75it/s]


padding
   revenue  gross_profit  ebitda  ebit  net_income  fcf  total_assets  \
0      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
1      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
2      NaN           NaN     NaN   NaN         NaN  NaN           NaN   
3      NaN           NaN     NaN   NaN         NaN  NaN           NaN   

   total_current_assets  total_liabilites  total_current_liabilities  \
0                   NaN               NaN                        NaN   
1                   NaN               NaN                        NaN   
2                   NaN               NaN                        NaN   
3                   NaN               NaN                        NaN   

   long_term_debt_p_assets  short_term_debt_p_assets  gross_profit_p  \
0                      NaN                       NaN             NaN   
1                      NaN                       NaN             NaN   
2                      NaN                       

In [ ]:
stock_df.pivot(index="ticker", columns="date", values="market_cap")

In [ ]:
it = iter(delta_set)
(stock, meta, macro), actuals = next(it)
(stock, meta, macro), actuals = next(it)

In [ ]:
stock

2009-02-02 00:00:00                   0.0
2009-02-03 00:00:00              0.059504
2009-02-04 00:00:00              0.060508
2009-02-05 00:00:00              0.045324
2009-02-06 00:00:00              0.104092
                                   ...   
short_term_debt_p_assets_q=-1        <NA>
gross_profit_p_q=-1             -0.145236
ebitda_p_q=-1                   -0.214431
ebit_p_q=-1                     -0.251064
net_income_p_q=-1                     NaN
Name: 000159.SZ, Length: 302, dtype: object

# Run the loop! (Like Odd-Geir Lademo)

![picture](https://drive.google.com/uc?id=1Y55gFQSi4Baovmi0kUQGhbgGOBTI03E7)


In [ ]:
def train_multivar(model, optimizer, loss_fn, data_train, data_val, one_hot_encoding, batch_number, forecast_window, epochs, device):

    # print(model)
    train_losses = []
    val_losses = []
    it = tqdm(range(epochs), disable=True)
    for epoch in it: 
        for run_type in ["train", "val"]:
            model.train(run_type == "train")

            if run_type == "train":
                optimizer.zero_grad()

                mu, sigma = 0, 0.1
                noise = np.random.normal(mu, sigma, data_train.shape)
                noise[:,-forecast_window:] = 0

                data_train = data_train + noise
                data_encoded = pd.concat([one_hot_encoding, data_train], axis=1, join="inner")

            else:
                data_encoded = pd.concat([one_hot_encoding, data_val], axis=1, join="inner")

            data_shuffled = torch.tensor(data_encoded.sample(frac=1).values, dtype=torch.float32)

            for batch in torch.split(data_shuffled, batch_number, dim=0):

                inputs = batch[:,:-forecast_window].to(device)
                actuals = batch[:,-forecast_window:].to(device)

                pred = model(inputs[:, 3:], inputs[:, 0:3])
                loss = loss_fn(pred, actuals)

                if run_type == "train":
                    train_losses.append(loss.item())
                    loss.backward()

                    optimizer.step()
                else:
                    val_losses.append(loss.item())

        it.set_postfix({"train_loss": np.mean(train_losses), "val_loss": np.mean(val_losses)})

    return train_losses, val_losses

IndentationError: ignored

In [ ]:
def run_multivar(hidden_dim):
    #Training loop params
    forecast_window = 16
    amount_of_time_series = 999
    length_of_time_series = 500
    epochs = 200
    batch_size = 111
    


    #_, TS_signal, one_hot_encoding  = time_series_df(amount_of_time_series,length_of_time_series,periods, horisontal_shift, vertical_shift, forecast_window)

    df_train = TS_signal.iloc[-int(amount_of_time_series/3):,:]
    df_val = TS_signal.iloc[:-int(amount_of_time_series/3),:]

    loss_fn = nn.L1Loss()
    
    model = MultivariateNetwork(length_of_time_series, one_hot_encoding.shape[1], forecast_window, hidden_dim)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    return train_multivar(model, optimizer, loss_fn, df_train, df_val, one_hot_encoding, batch_size, forecast_window, epochs, device)